# Entraînement du modèle de classification des prunes africaines

Ce notebook utilise les fonctions existantes dans le dépôt pour entraîner le modèle de classification des prunes africaines.

## 1. Configuration de l'environnement

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Ajouter le répertoire parent au chemin pour pouvoir importer nos modules
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importer nos modules personnalisés
from data.data_preprocessing import load_and_prepare_two_stage_data
from models.model_architecture import get_model
from scripts.train_two_stage import train_model, evaluate_model, plot_training_history

# Définir les chemins des données
DATA_ROOT = "../data/raw"  # Chemin vers le répertoire de données brutes
PLUM_DATA_DIR = os.path.join(DATA_ROOT, "plums")  # Sous-dossier pour les prunes
NON_PLUM_DATA_DIR = os.path.join(DATA_ROOT, "non_plums")  # Sous-dossier pour les non-prunes
MODELS_DIR = "../models/saved"  # Répertoire pour sauvegarder les modèles entraînés

# Créer les répertoires s'ils n'existent pas
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(PLUM_DATA_DIR, exist_ok=True)
os.makedirs(NON_PLUM_DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Définir les paramètres d'entraînement
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_WORKERS = 4
LEARNING_RATE = 0.001
NUM_EPOCHS = 25
EARLY_STOPPING_PATIENCE = 7
RANDOM_SEED = 42

# Fixer les seeds pour la reproductibilité
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

## 2. Chargement des données

Utilisons la fonction `load_and_prepare_two_stage_data` du module `data_preprocessing` pour charger les données.

In [ ]:
# Vérifier si les répertoires de données existent et contiennent des images
def check_data_availability():
    # Vérifier le répertoire des prunes
    plum_classes = [d for d in os.listdir(PLUM_DATA_DIR) if os.path.isdir(os.path.join(PLUM_DATA_DIR, d))]
    if not plum_classes:
        print(f"Aucune classe de prune trouvée dans {PLUM_DATA_DIR}. Veuillez ajouter des données.")
        return False
    
    # Vérifier le répertoire des non-prunes
    non_plum_dir = os.path.join(NON_PLUM_DATA_DIR, "non_plum")
    if not os.path.exists(non_plum_dir):
        print(f"Le répertoire {non_plum_dir} n'existe pas. Veuillez créer ce répertoire et y ajouter des images.")
        return False
    
    return True

# Vérifier la disponibilité des données
data_available = check_data_availability()

if data_available:
    try:
        # Charger et préparer les données pour les deux étapes
        print("Chargement des données pour les deux étapes...")
        (detection_train_loader, detection_val_loader, detection_test_loader, detection_class_names), \
        (classification_train_loader, classification_val_loader, classification_test_loader, classification_class_names) = \
            load_and_prepare_two_stage_data(
                PLUM_DATA_DIR, 
                NON_PLUM_DATA_DIR,
                batch_size=BATCH_SIZE, 
                img_size=IMG_SIZE,
                num_workers=NUM_WORKERS
            )
        
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
    except Exception as e:
        print(f"Erreur lors du chargement des données: {e}")
else:
    print("Veuillez d'abord ajouter des données dans les répertoires appropriés.")

## 3. Création des modèles

Utilisons la fonction `get_model` du module `model_architecture` pour créer les modèles.

In [ ]:
if data_available and 'detection_class_names' in locals() and 'classification_class_names' in locals():
    # Créer le modèle de détection
    detection_model = get_model(
        model_name='lightweight', 
        num_classes=len(detection_class_names), 
        base_model='mobilenet_v2', 
        pretrained=True
    )
    
    # Créer le modèle de classification
    classification_model = get_model(
        model_name='standard', 
        num_classes=len(classification_class_names),
        base_model='resnet18', 
        pretrained=True
    )
    
    # Afficher les informations sur les modèles
    print("=== Modèle de détection ===")
    print(f"Type: {detection_model.__class__.__name__}")
    print(f"Informations: {detection_model.get_model_info()}")
    
    print("\n=== Modèle de classification ===")
    print(f"Type: {classification_model.__class__.__name__}")
    print(f"Informations: {classification_model.get_model_info()}")
    
    # Déplacer les modèles sur le device
    detection_model = detection_model.to(device)
    classification_model = classification_model.to(device)

## 4. Entraînement du modèle de détection

Utilisons la fonction `train_model` du module `train_two_stage` pour entraîner le modèle de détection.

In [ ]:
if data_available and 'detection_train_loader' in locals() and 'detection_val_loader' in locals() and 'detection_model' in locals():
    try:
        print("=== Entraînement du modèle de détection ===\n")
        
        # Définir la fonction de perte et l'optimiseur
        detection_criterion = nn.CrossEntropyLoss()
        detection_optimizer = optim.Adam(detection_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        detection_scheduler = ReduceLROnPlateau(detection_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de détection
        detection_history = train_model(
            detection_model, 
            detection_train_loader, 
            detection_val_loader, 
            detection_criterion, 
            detection_optimizer, 
            detection_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(detection_history, save_dir=MODELS_DIR, model_name="detection")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de détection: {e}")

## 5. Entraînement du modèle de classification

Utilisons la fonction `train_model` du module `train_two_stage` pour entraîner le modèle de classification.

In [ ]:
if data_available and 'classification_train_loader' in locals() and 'classification_val_loader' in locals() and 'classification_model' in locals():
    try:
        print("=== Entraînement du modèle de classification ===\n")
        
        # Définir la fonction de perte et l'optimiseur
        classification_criterion = nn.CrossEntropyLoss()
        classification_optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
        
        # Scheduler pour ajuster le learning rate
        classification_scheduler = ReduceLROnPlateau(classification_optimizer, mode='min', factor=0.1, patience=3, verbose=True)
        
        # Entraîner le modèle de classification
        classification_history = train_model(
            classification_model, 
            classification_train_loader, 
            classification_val_loader, 
            classification_criterion, 
            classification_optimizer, 
            classification_scheduler, 
            device, 
            num_epochs=NUM_EPOCHS, 
            early_stopping_patience=EARLY_STOPPING_PATIENCE,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
        
        # Tracer les courbes d'entraînement
        plot_training_history(classification_history, save_dir=MODELS_DIR, model_name="classification")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle de classification: {e}")

## 6. Évaluation des modèles

Utilisons la fonction `evaluate_model` du module `train_two_stage` pour évaluer les modèles.

In [ ]:
# Évaluer le modèle de détection
if data_available and 'detection_test_loader' in locals() and 'detection_class_names' in locals():
    try:
        print("=== Évaluation du modèle de détection ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth')))
        detection_model = detection_model.to(device)
        
        # Évaluer le modèle
        detection_criterion = nn.CrossEntropyLoss()
        detection_metrics = evaluate_model(
            detection_model, 
            detection_test_loader, 
            detection_criterion, 
            device, 
            detection_class_names,
            save_dir=MODELS_DIR,
            model_name="detection"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de détection: {e}")

In [ ]:
# Évaluer le modèle de classification
if data_available and 'classification_test_loader' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Évaluation du modèle de classification ===\n")
        
        # Charger le meilleur modèle (selon l'accuracy)
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth')))
        classification_model = classification_model.to(device)
        
        # Évaluer le modèle
        classification_criterion = nn.CrossEntropyLoss()
        classification_metrics = evaluate_model(
            classification_model, 
            classification_test_loader, 
            classification_criterion, 
            device, 
            classification_class_names,
            save_dir=MODELS_DIR,
            model_name="classification"
        )
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle de classification: {e}")

## 7. Sauvegarde du modèle à deux étapes

Créons et sauvegardons le modèle à deux étapes complet.

In [ ]:
import json
import time
from models.model_architecture import TwoStageModel

# Créer et sauvegarder le modèle à deux étapes
if data_available and 'detection_class_names' in locals() and 'classification_class_names' in locals():
    try:
        print("=== Création du modèle à deux étapes ===\n")
        
        # Charger les meilleurs modèles
        detection_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'detection_best_acc.pth')))
        classification_model.load_state_dict(torch.load(os.path.join(MODELS_DIR, 'classification_best_acc.pth')))
        
        # Créer le modèle à deux étapes
        two_stage_model = TwoStageModel(detection_model, classification_model, detection_threshold=0.7)
        
        # Sauvegarder les informations du modèle
        model_info = {
            'detection_classes': detection_class_names,
            'classification_classes': classification_class_names,
            'model_info': two_stage_model.get_model_info(),
            'img_size': IMG_SIZE,
            'date_created': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(os.path.join(MODELS_DIR, 'two_stage_model_info.json'), 'w') as f:
            json.dump(model_info, f, indent=4)
        
        print("Modèle à deux étapes créé et informations sauvegardées.")
        print(f"Classes de détection: {detection_class_names}")
        print(f"Classes de classification: {classification_class_names}")
        print(f"Informations du modèle: {two_stage_model.get_model_info()}")
    except Exception as e:
        print(f"Erreur lors de la création du modèle à deux étapes: {e}")

## 8. Conclusion

Dans ce notebook, nous avons utilisé les fonctions existantes des modules `data_preprocessing`, `model_architecture` et `train_two_stage` pour :
1. Charger les données
2. Créer les modèles de détection et de classification
3. Entraîner les modèles
4. Évaluer les performances
5. Créer et sauvegarder le modèle à deux étapes complet

Le modèle est maintenant prêt à être testé sur de nouvelles images dans le notebook suivant.